# Create Dataset

In [ ]:
import numpy as np

class generateDataset:
  def __init__(self, inputs, set_size):
    self.inputs=inputs
    self.set_size = set_size

  def XOR(self): #xor function on 2 inputs
    X, y = [],[]
    for i in range(self.set_size):
      gen = [np.random.randint(0,2) for i in range(2)]
      X.append(gen)
      y.append(gen[0] ^ gen[1])
    X,y = np.array(X), np.array(y)
    X,y = X.reshape(self.set_size, self.inputs), y.reshape(self.set_size, 1)
    return X,y

  def ODD_EVEN(self): #odd even function on any number of inputs
    X, y = [],[]
    for i in range(self.set_size):
        gen = [np.random.randint(0,2) for i in range(self.inputs)]
        X.append(gen)
        y.append(sum(gen)%2)
    X,y = np.array(X), np.array(y)
    X,y = X.reshape(self.set_size, self.inputs), y.reshape(self.set_size, 1)
    return X,y #odd is 1 even is 0

xor = generateDataset(2,100).XOR()
even_odd = generateDataset(5,100).ODD_EVEN()

print(str(even_odd[0]) + "\n\n" + str(even_odd[1]))

# Create Artifical Neural Net Class

In [ ]:
class ANN:
    #Initialize network with hyperparamaters and weights, biases
    def __init__(self, lr, epochs, inputs, hidden1, hidden2, X, y):
        self.lr = lr
        self.epochs = epochs
        #create layers
        self.w1 = np.random.randn(inputs, hidden1) * np.sqrt(1/inputs)
        self.w2 = np.random.randn(hidden1, hidden2) * np.sqrt(1/hidden1)
        self.w3 = np.random.randn(hidden2, 1) * np.sqrt(1/hidden2)
        self.b1 = np.zeros((1, hidden1))
        self.b2 = np.zeros((1, hidden2))
        self.b3 = np.zeros((1, 1))
        #load datasets
        self.X = X
        self.y = y

    # Define sigmoid and sigmoid derivitive for use in forward and backward propagation.
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def sigmoid_derivative(self, x):
        return x*(1-x)

    # Forward propagation as dot product of each layer with sigmoid activation function.
    def forward_pass(self, X):
        self.a1 = np.dot(X, self.w1) + self.b1
        self.o1 = self.sigmoid(self.a1)
        self.a2 = np.dot(self.a1, self.w2) + self.b2
        self.o2 = self.sigmoid(self.a2)
        self.a3 = np.dot(self.a2, self.w3) + self.b3
        self.o3 = self.sigmoid(self.a3)
        return self.o3

    # Backpropagation with
    def backprop(self, X, y):
        #Find what error each layer is responsible for
        self.error = self.o3 - y
        self.d3 = np.multiply(self.error, self.sigmoid_derivative(self.o3))
        self.d2 = np.multiply(np.dot(self.d3, self.w3.T), self.sigmoid_derivative(self.o2))
        self.d1 = np.multiply(np.dot(self.d2, self.w2.T), self.sigmoid_derivative(self.o1))

        #Change weights and biases based on error from each layer
        self.w3 -= self.lr * np.dot(self.a2.T, self.d3)
        self.w2 -= self.lr * np.dot(self.a1.T, self.d2)
        self.w1 -= self.lr * np.dot(X.T, self.d1)

        self.b3 -= self.lr * np.sum(self.d3, axis=0)
        self.b2 -= self.lr * np.sum(self.d2, axis=0)
        self.b1 -= self.lr * np.sum(self.d1, axis=0)

    def Train(self):
        for i in range(self.epochs):
            self.forward_pass(self.X)
            self.backprop(self.X, self.y)
            if i % 1000 == 0:
                print("Epoch: ", i)

# Train and test the model on the dataset.

In [ ]:
# Create models
xor_net = ANN(.1, 10000, 2,3,2,xor[0], xor[1])
eo_net = ANN(.1,10000,5,10,4,even_odd[0], even_odd[1])

#Train models
print("Training xor:")
xor_net.Train()
print("Training even_odd:")
eo_net.Train()

#Test case for XOR
print("Xor test case:")
print(xor_net.forward_pass([0,0]))
#Test case for EVEN_ODD
print("Even_odd test case:")
print(eo_net.forward_pass([1,1,1,1,0]))

